In [1]:
from unlabeled_extrapolation.models import bit_resnet, vit_model
from unlabeled_extrapolation.utils import utils
import importlib
importlib.reload(bit_resnet)
importlib.reload(vit_model)
importlib.reload(utils)

<module 'unlabeled_extrapolation.utils.utils' from '/juice/scr/ananya/cifar_experiments/transfer_learning/unlabeled_extrapolation/utils/utils.py'>

In [2]:
def get_layers_freeze_test(model):
    print('num params before freezing: ', utils.count_parameters(model, trainable=True))
    print(model.get_layers()[1])
    print(len(model.get_layers()))
    model.freeze_bottom_k(k=1)
    print('num params after freezing 1: ', utils.count_parameters(model, trainable=True))
    model.freeze_bottom_k(k=2)
    print('num params after freezing 2: ', utils.count_parameters(model, trainable=True))

In [3]:
# Bit-resnet (get layers) for ResNet-50 and ResNet-101

resnet50_checkpoint_path = "/u/scr/ananya/simclr_weights/BiT-M-R50x1.npz"
resnet50 = bit_resnet.BitResNet(model_name='BiT-M-R50x1', checkpoint_path=resnet50_checkpoint_path)
get_layers_freeze_test(resnet50)

resnet101_checkpoint_path = "/u/scr/ananya/simclr_weights/BiT-M-R101x1.npz"
resnet101 = bit_resnet.BitResNet(model_name='BiT-M-R101x1', checkpoint_path=resnet101_checkpoint_path)
get_layers_freeze_test(resnet101)

num params before freezing:  68256659
('conv-block-0', Sequential(
  (unit01): PreActBottleneck(
    (gn1): GroupNorm(32, 64, eps=1e-05, affine=True)
    (conv1): StdConv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (gn2): GroupNorm(32, 64, eps=1e-05, affine=True)
    (conv2): StdConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (gn3): GroupNorm(32, 64, eps=1e-05, affine=True)
    (conv3): StdConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (relu): ReLU(inplace=True)
    (downsample): StdConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  )
  (unit02): PreActBottleneck(
    (gn1): GroupNorm(32, 256, eps=1e-05, affine=True)
    (conv1): StdConv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (gn2): GroupNorm(32, 64, eps=1e-05, affine=True)
    (conv2): StdConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (gn3): GroupNorm(32, 64, eps=1e-05, affine=True)
    (conv3)

In [4]:
# DINO
model = vit_model.VitModel(model_name='dino_vitb16')
get_layers_freeze_test(model)

Downloading: "https://github.com/facebookresearch/dino/archive/main.zip" to /sailhome/ananya/.cache/torch/hub/main.zip


num params before freezing:  85798656
('empty_ln_pre', Module())
54
num params after freezing 1:  85208064
num params after freezing 2:  85208064


In [5]:
# Timm ViT-S
model = vit_model.VitModel(model_name='timm.vit_small_patch16_224')
get_layers_freeze_test(model)

num params before freezing:  22050664
('empty_ln_pre', Module())
54
num params after freezing 1:  21755368
num params after freezing 2:  21755368


In [ ]:
# Conv-next

